In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [2]:
words = open('names.txt', 'r').read().splitlines()

chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
block_size = 3

def build_dataset(words):

  X, Y = [], []
  for w in words:
    #print(w)
    context = [0]* block_size

    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context),'--->', itos[ix])
      context = context[1:]+[ix]

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y



import random
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))


Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

print(Xtr[:5])

torch.Size([182472, 3]) torch.Size([182472])
torch.Size([22928, 3]) torch.Size([22928])
torch.Size([22746, 3]) torch.Size([22746])
tensor([[ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 14],
        [ 1, 14,  3],
        [14,  3,  8]])


In [ ]:
#building MLP
n_embd = 10 #dimensionality of character embedding vectors
n_hidden = 200 #no. of neurons in hidden layer of MLP

C = torch.rand((vocab_size,n_embd))
W1 = torch.randn((n_embd*block_size,n_hidden)) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden) * 0.01  #b1 redundant because of batchnormalisation bias
W2 = torch.randn((n_hidden,vocab_size)) *0.1
b2 = torch.randn(vocab_size) * 0.1


bngain = torch.randn((1, n_hidden))*0.1+1.0
bnbias = torch.randn((1, n_hidden))*0.1  #non-zero to unmask errors in calculating gradients w.r.t biases.

#bnmean_running = torch.zeros((1, n_hidden))
#bngain_running = torch.ones((1, n_hidden))

parameters = [C, W1 , W2, b2, bngain, bnbias]


for p in parameters:
  p.requires_grad = True

In [7]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,))
Xb, Yb = Xtr[ix], Ytr[ix]


